# tohiview v1.0.1

This is a updated tutorial of visualizing hierarchical protein network modules, via a script that intefacing the DDOT python package (v1.0.1) and the HiView server (v2.6)

Author: Fan Zheng  
Date: 08/04/2020

If the DDOT package has been installed and all dependencies are satisfied, all you need is the upload script `tohiview.py`, and a few input files of that script. This tutorial will walk over the creation of hierarchical models and their visualization in HiView.   

In [20]:
username = 'fzheng' # replace with your username

In [16]:
import getpass
passwd = getpass.getpass("Passwd here: ")

Passwd here:  ········


The available options of the upload script are listed below. Many options are available, but only `--ont`, `--hier_name`, `--ndex_acount` are required.

`--ont` should be a 3-col file defined in DDOT, which represent parent, child and type of the relationship (see details here).  
`--hier_name` is just a string to label the files.   
`--ndex_acount` contains 3 strings, the server name (http://www.ndexbio.org for all external users), a username and a password.

In [15]:
%%bash 

python ../../ddot/tohiview.py -h

usage: tohiview.py [-h] --ont ONT --hier_name HIER_NAME
                   [--subnet_size SUBNET_SIZE SUBNET_SIZE]
                   [--node_attr NODE_ATTR] [--subnet_links SUBNET_LINKS]
                   [--rf_score RF_SCORE] [--gene_attr GENE_ATTR]
                   [--term_2_uuid TERM_2_UUID]
                   [--visible_cols [VISIBLE_COLS [VISIBLE_COLS ...]]]
                   [--max_num_edges MAX_NUM_EDGES] [--col_color COL_COLOR]
                   [--col_label COL_LABEL]
                   [--ndex_account NDEX_ACCOUNT NDEX_ACCOUNT NDEX_ACCOUNT]
                   [--rename RENAME] [--skip_main]

optional arguments:
  -h, --help            show this help message and exit
  --ont ONT             ontology file, 3 col table
  --hier_name HIER_NAME
                        name of the hierarchy
  --subnet_size SUBNET_SIZE SUBNET_SIZE
                        minimum and maximum term size to show network support
  --node_attr NODE_ATTR
                        table file for attribu

In /cellar/users/f6zheng/anaconda2/envs/ddot/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /cellar/users/f6zheng/anaconda2/envs/ddot/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /cellar/users/f6zheng/anaconda2/envs/ddot/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


# 1. A simple hierarchy

We will first create and upload a decoy hierarchy.

In [31]:
d = './data'
df = pd.read_csv(d + '/decoy.txt', sep='\t', header=None)
df

,0,1,2
0,ROOT,Coarse-1,default
1,ROOT,Coarse-2,default
2,Coarse-1,Fine-1,default
3,Coarse-1,Fine-2,default
4,Coarse-1,Fine-3,default
5,Coarse-2,Fine-3,default
6,Coarse-2,Fine-4,default
7,Fine-1,A,gene
8,Fine-1,B,gene
9,Coarse-1,C,gene


In [22]:
%%bash -s "$username", "$passwd"

python ../../ddot/tohiview.py --ont ./data/decoy.txt --hier_name decoy --ndex_account http://www.ndexbio.org $1 $2

Creating NdexGraph

Uploading to NDEx


In /cellar/users/f6zheng/anaconda2/envs/ddot/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /cellar/users/f6zheng/anaconda2/envs/ddot/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /cellar/users/f6zheng/anaconda2/envs/ddot/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
Traceback (most recent call last):
  File "../../ddot/tohiview.py", line 241, in <module>
    main_uuid = upload_main_hierarchy(ont, args.hier_name, term_uuid, vis_cols)
  File "../../ddot/tohiview.py", line 148, in upload_main_hierarchy
    verbose=True)
  File "/cellar/users/f6zheng/anaconda2/envs/ddot/lib/python3.6/site-packages/ddot/Ontology.py", lin

CalledProcessError: Command 'b'\npython ../../ddot/tohiview.py --ont ./data/small-1.txt --hier_name small-1 --ndex_account http://www.ndexbio.org $1 $2\n'' returned non-zero exit status 1.

# 2. Adding integrated networks to communities

HiView is a powerful platform to display multiscale communities in a network. It is often of interest to visualize edges in the source network that support a community.   Precisely, for a source network $G = (V, E)$, a subnetwork of a community $s$ is defined as $G_s = (V_s, E_s)$, where $\forall e = (u,v) \in E_s$, $u,v \in V_s$.  


This is achieved by the `--score` argument. It is a tab seperated file with three columns: `geneA`,`geneB` and `score`. We recommend having the values of score reside within (0,1). 

## "score" of a community.   

This is a concept specific to certain community detection algorithms, e.g. CliXO. CliXO takes a weighted graph as the input, and iterate community detection at different thresholds. Thus, each community in CliXO is associated with a "score".

By default, edges in a subnetwork have a uniform color in HiView. However, if communities are associated with scores, the edges will be shown with a discrete color map (which often visually highlights the community structures), determined by the score of the community itself, and the score(s) of its children community(ies). This can be achieved by adding a 4-th column to the file for the `--ont` argument, as in the following example:

# 3. Adding multiple evidence networks to systems

In addition to a single master network, it is also possible to overlay more networks supporting a community and visualize them HiView. For example, if the master network is the result of integrating multiple datasets, it is often of interest to visualize the interactions in these datasets (jointly or separately).

## Large networks

Large scale networks are often bottlenecks of the speed of uploading and HiView visualization (we are working on improving that). To reduce overhead, subnetwork uploading can be disabled for large communities, while still being enabled for smaller communities. 

It is achieved by the `--subnet_size` argument, which takes two integers, specifying the lower and upper bound of community sizes for which upload of the integrated subnetworks is enabled.

Similarly `--evinet_size` takes one integer, and for communities larger than this threshold, upload of evidence networks will be disabled.

We require `subnet_size[0] < evinet_size <= subnet_size[1]`.

# 4. Reuse uploaded subnetworks

After uploading a hierarchy, a file starting with `term_2_uuid` will be writing to the working directory. This file describe the mapping between community names and community subnetworks. 

This file can also be later used as the input of `--term_2_uuid` argument, so subnetworks can be shared across different hierarchical models.

# 5. Control the information displayed in HiView

# 6. Delete a HiView session from NDEx account

After a upload has been finished, the script creates a folder (network set) on the NDEx account containing one network of the hierarchical model (to be used in the model view), as well as many subnetworks (to be used in the data view). With the button `Delete Network Set`, the set and all networks in this set can be deleted.

**Warning**: note that if subnetworks are shared across models (see Section 4 above), the deleting operation could affect other models (unwanted).